In [55]:
import pandas as pd

output = pd.read_parquet(r"C:\Users\ChristopherCato\OneDrive - clarity-dx.com\code\bph\workcomp-rates\core\data\commercial_rates_GA.parquet")



In [69]:
uhc_provider_list = pd.read_parquet(r"C:\Users\ChristopherCato\Downloads\providers_uhc_georgia_20250801_20250828_200746.parquet")
print(uhc_provider_list.columns)
print(len(uhc_provider_list))

Index(['provider_group_id', 'npi', 'tin_type', 'tin_value',
       'reporting_entity_name', 'reporting_entity_type', 'last_updated_on',
       'version'],
      dtype='object')
17855


In [56]:
len(output)

12150

In [37]:
import requests

def address_to_latlon_cbsa(addr: str):
    base = "https://geocoding.geo.census.gov/geocoder/geographies/onelineaddress"
    params = {
        "address": addr,
        "benchmark": "Public_AR_Current",
        "vintage": "Current_Current",             # required for geographies
        # CBSA layers (either names or IDs 93=Metro, 91=Micro)
        "layers": "Metropolitan Statistical Areas,Micropolitan Statistical Areas",
        "format": "json",
    }
    j = requests.get(base, params=params, timeout=30).json()
    matches = j.get("result", {}).get("addressMatches", [])
    if not matches:
        return {"error": "no_match"}

    m = matches[0]
    lon, lat = m["coordinates"]["x"], m["coordinates"]["y"]

    # Try CBSA directly from this response
    geos = j["result"].get("geographies", {})
    msa = geos.get("Metropolitan Statistical Areas", []) or geos.get("Micropolitan Statistical Areas", [])

    # If CBSA not present, fall back: look up by coordinates
    if not msa:
        coord_url = "https://geocoding.geo.census.gov/geocoder/geographies/coordinates"
        coord_params = {
            "x": lon, "y": lat,
            "benchmark": "Public_AR_Current",
            "vintage": "Current_Current",
            "layers": "93,91",   # Metro, Micro by ID
            "format": "json",
        }
        j2 = requests.get(coord_url, params=coord_params, timeout=30).json()
        geos2 = j2.get("result", {}).get("geographies", {})
        msa = geos2.get("Metropolitan Statistical Areas", []) or geos2.get("Micropolitan Statistical Areas", [])

    cbsa = None
    if msa:
        cbsa = {"cbsa_geoid": msa[0]["GEOID"], "cbsa_name": msa[0]["NAME"]}

    return {"lat": lat, "lon": lon, "cbsa": cbsa}

# Example
#462 GRIDER ST        BUFFALO    NY   142153098
print(address_to_latlon_cbsa("462 GRIDER ST, BUFFALO, NY 142153098"))

# print(address_to_latlon_cbsa("4600 Silver Hill Rd, Washington, DC 20233"))


{'lat': 42.926457555011, 'lon': -78.829223514588, 'cbsa': {'cbsa_geoid': '15380', 'cbsa_name': 'Buffalo-Cheektowaga, NY Metro Area'}}


In [46]:
output.columns

Index(['rate', 'negotiated_type', 'billing_class', 'service_codes',
       'billing_code', 'billing_code_type', 'code_desc', 'name',
       'negotiation_arrangement', 'payer', 'payer_type', 'rate_updated_on',
       'prov_npi', 'tin_type', 'tin_value', 'GA_PROF_MAR', 'GA_OP_MAR',
       'GA_ASC_MAR', 'org_name', 'status', 'primary_taxonomy_code',
       'primary_taxonomy_desc', 'address_purpose', 'address_type', 'address_1',
       'address_2', 'city', 'state', 'postal_code', 'country_code',
       'telephone_number', 'fax_number', 'lat', 'lon', 'cbsa_geoid',
       'cbsa_name', 'geocoding_error', 'medicare_prof', 'state_up',
       'billing_code_norm', 'state_wage_index_avg', 'opps_weight', 'opps_si',
       'opps_short_desc', 'asc_pi', 'asc_nat_rate', 'asc_short_desc',
       'medicare_opps_mar_national', 'medicare_asc_mar_national',
       'opps_adj_factor_stateavg', 'asc_adj_factor_stateavg',
       'medicare_opps_mar_stateavg', 'medicare_asc_mar_stateavg',
       'procedure_set', 

In [39]:
print(f"unique procedure_set: {output['procedure_set'].unique()}")

print(f"unique payer: {output['payer'].unique()}")


print(f"unique org_name: {output['org_name'].unique()}")

print(f"unique cbsa_geoid: {output['cbsa_geoid'].unique()}")

unique procedure_set: ['Radiology']
unique payer: ['UnitedHealthcare of Georgia Inc.']
unique org_name: ['BEAVER VALLEY AMBULATORY SURGERY CENTER, LLC' 'MIAMI ASC LP'
 'KENTUCKIANA PAIN SPECIALIST AMBULATORY SURGERY CENTER'
 'KURT G VERNON MD PA' 'UINTAH BASIN MEDICAL CENTER'
 'ADVANCED SURGERY CENTER OF ORLANDO LLC' 'THE MEMORIAL HOSPITAL'
 'CENTURY CITY ENDOSCOPY LLC' 'CENTURY CITY ENDOSCOPY, LLC'
 'WILMINGTON SURGERY CENTER, LP' 'HEALTHQARE SERVICES ASC LLC'
 'PIEDMONT SURGERY CENTER' 'nan' 'VISION QUEST LASER CENTER, LTD'
 'SURGEYECARE, INC.' 'PRECISION AMBULATORY SURGERY CENTER, LLC'
 'HARLINGEN SURGICAL CENTER, LLC'
 'SOUTHERN CRESCENT REHABILITATION AND RETIREMENT COMMUNITY, INC'
 'LISA RENFRO SURGERY CENTER LLC' 'DESERT PEAKS SURGERY CENTER LLC'
 'BLUE MOUNTAIN HOSPITAL DISTRICT' 'GEORGIA INTERVENTIONAL PAIN - I, LLC'
 'CENTER FOR MINIMALLY INVASIVE SURGERY PLLC'
 'RETINA CONSULTANTS SURGERY CENTER'
 'CHRISTUS SANTA ROSA HEALTH CARE CORPORATION'
 'MOHS AND SKIN SURGERY CENTER, 

In [40]:
print(output[['address_1', 'city', 'state', 'postal_code']].sample(5))

                     address_1        city state postal_code
4963       500 LAKE CONCORD RD     CONCORD    NC   280252926
11133  9439 NORTH BEACH STREET  FORT WORTH    TX       76244
5500             462 GRIDER ST     BUFFALO    NY   142153021
8317           20 COMMUNITY DR      EASTON    PA   180452658
1995         2002 MEDICAL PKWY   ANNAPOLIS    MD   214013046


In [58]:
print(f"unique state: {output['state'].unique()}")
print(f"unique postal_code: {output['postal_code'].unique()}")
output_ga = output[output['state'].isin(['GA'])]
print(f"len(output_ga) after filtering states: {len(output_ga)}")
print(f"unique org_name in output_ga: {output_ga['org_name'].unique()}")


unique state: ['PA' 'FL' 'KY' 'NC' 'UT' 'CO' 'CA' 'VA' 'GA' 'nan' 'TX' 'MD' 'NM' 'OR'
 'WA' 'NJ' 'ID' 'MS' 'NY' 'MO' 'MN' 'IN']
unique postal_code: ['150612489' '331342442' '402412002' '27518' '275262620' '840662336'
 '840783252' '328194707' '81625' '816252019' '900672001' '284016443'
 '222015792' '312101395' 'nan' '752314107' '76244' '785508209' '302534733'
 '214013046' '880118191' '978452009' '97869' '301616019' '983352095'
 '984054499' '077391208' '781304194' '782514498' '786667533' '210936017'
 '932409529' '93240' '344297622' '83201' '216013888' '396672022' '39667'
 '142153021' '142153098' '301651537' '631226425' '560015066' '338052442'
 '221822710' '196101986' '468041537' '462021221' '180452658' '082211036'
 '281055012' '187044968' '214013159' '280252926' '837048731']
len(output_ga) after filtering states: 540
unique org_name in output_ga: ['PIEDMONT SURGERY CENTER'
 'SOUTHERN CRESCENT REHABILITATION AND RETIREMENT COMMUNITY, INC'
 'GEORGIA INTERVENTIONAL PAIN - I, LLC' 'ROME SURG

In [59]:
optim = pd.read_parquet(r"C:\Users\ChristopherCato\OneDrive - clarity-dx.com\code\bph\uhc_tic\optim_health\combined_df.parquet")


In [60]:
optim.columns

Index(['Unnamed: 0', 'provider_group_id', 'npi', 'tin_type', 'tin_value',
       'reporting_entity_name', 'reporting_entity_type', 'last_updated_on_x',
       'version_x', 'provider_reference_id', 'uhc_rate', 'negotiated_type',
       'billing_class', 'expiration_date', 'service_codes', 'billing_code',
       'billing_code_type', 'description', 'name', 'negotiation_arrangement',
       'last_updated_on_y', 'version_y', 'provider_name', 'credentials',
       'gender', 'enumeration_date', 'last_updated', 'addresses',
       'primary_specialty', 'secondary_specialties', 'provider_type',
       'metadata', 'city', 'country', 'fax', 'phone', 'purpose', 'state',
       'street', 'type', 'full_zip', 'service_zip', 'ZIP', 'CBSA',
       'USPS_ZIP_PREF_CITY', 'USPS_ZIP_PREF_STATE', 'RES_RATIO', 'BUS_RATIO',
       'OTH_RATIO', 'TOT_RATIO', 'CBSA Code', 'Metropolitan Division Code',
       'CSA Code', 'CBSA Title', 'Metropolitan/Micropolitan Statistical Area',
       'Metropolitan Division Title

In [68]:
print(optim['state'].unique())
print(optim['provider_name'].unique())
print(str(optim['npi'].unique()))
print(optim['primary_specialty'].unique())


['GA']
["{'first': '', 'last': '', 'middle': '', 'suffix': ''}"]
[1.70089975e+09]
['Physical Medicine & Rehabilitation']


In [54]:
print(f"unique npi: {output['prov_npi'].unique()}")

unique npi: ['1962531988' '1851530554' '1386213460' '1902020522']


In [52]:
print(f"unique org_name: {output['org_name'].unique()}")

unique org_name: ['PIEDMONT SURGERY CENTER'
 'SOUTHERN CRESCENT REHABILITATION AND RETIREMENT COMMUNITY, INC'
 'GEORGIA INTERVENTIONAL PAIN - I, LLC' 'ROME SURGICAL CENTER, P.C.']


In [6]:
print(f"unique primary_taxonomy_desc: {output['primary_taxonomy_desc'].unique()}")

unique primary_taxonomy_desc: ['Clinic/Center, Ambulatory Surgical'
 'Internal Medicine, Gastroenterology' 'Clinic/Center, Endoscopy'
 'Clinic/Center, End-Stage Renal Disease (ESRD) Treatment'
 'General Acute Care Hospital, Critical Access'
 'Clinic/Center, Multi-Specialty' 'nan'
 'Clinic/Center, Ophthalmologic Surgery' 'Long Term Care Hospital'
 'Nursing Facility/Intermediate Care Facility'
 'Clinic/Center, Rural Health' 'Surgery' 'General Acute Care Hospital'
 'Skilled Nursing Facility' 'Clinic/Center' 'Anesthesiology'
 'Clinic/Center, Ambulatory Fertility Facility']


In [8]:
# drop unwanted primary_taxonomy_desc

desc_to_drop = [
    'Gastroenterology',
    'Endoscopy',
    'Ambulatory Fertility Center',
    'Ophthalmologic Surgery',
    'End-Stage Renal Disease (ESRD) Treatment',
    'Anesthesiology'
]
print(f"len(output) before dropping: {len(output)}")
output = output[~output['primary_taxonomy_desc'].isin(desc_to_drop)]
print(f"len(output) after dropping: {len(output)}")

len(output) before dropping: 12312
len(output) after dropping: 12150


In [ ]:

display(output.head())


,rate,negotiated_type,billing_class,service_codes,billing_code,billing_code_type,code_desc,name,negotiation_arrangement,payer,...,asc_short_desc,medicare_opps_mar_national,medicare_asc_mar_national,opps_adj_factor_stateavg,asc_adj_factor_stateavg,medicare_opps_mar_stateavg,medicare_asc_mar_stateavg,procedure_set,procedure_class,procedure_group
0,18.15,negotiated,professional,[],70030,CPT,"Radiologic examination, eye, for detection of ...",RADIOLOGIC EXAMINATION EYE DETECT FOREIGN BODY,ffs,UnitedHealthcare of Georgia Inc.,...,None,88.045471,NaN,0.926806,0.939005,81.601074,NaN,Radiology,Head and Neck Imaging,Skull and Brain
1,18.15,negotiated,professional,[],70030,CPT,"Radiologic examination, eye, for detection of ...",RADIOLOGIC EXAMINATION EYE DETECT FOREIGN BODY,ffs,UnitedHealthcare of Georgia Inc.,...,None,88.045471,NaN,0.926806,0.939005,81.601074,NaN,Radiology,Head and Neck Imaging,Skull and Brain
2,18.15,negotiated,professional,[],70030,CPT,"Radiologic examination, eye, for detection of ...",RADIOLOGIC EXAMINATION EYE DETECT FOREIGN BODY,ffs,UnitedHealthcare of Georgia Inc.,...,None,88.045471,NaN,0.926806,0.939005,81.601074,NaN,Radiology,Head and Neck Imaging,Skull and Brain
3,25.08,negotiated,professional,[],70030,CPT,"Radiologic examination, eye, for detection of ...",RADIOLOGIC EXAMINATION EYE DETECT FOREIGN BODY,ffs,UnitedHealthcare of Georgia Inc.,...,None,88.045471,NaN,0.926806,0.939005,81.601074,NaN,Radiology,Head and Neck Imaging,Skull and Brain
4,25.08,negotiated,professional,[],70030,CPT,"Radiologic examination, eye, for detection of ...",RADIOLOGIC EXAMINATION EYE DETECT FOREIGN BODY,ffs,UnitedHealthcare of Georgia Inc.,...,None,88.045471,NaN,0.926806,0.939005,81.601074,NaN,Radiology,Head and Neck Imaging,Skull and Brain


In [9]:
# You can use the `fastparquet` or `pyarrow` engine with pandas to append to a Parquet file,
# but true "append" mode is only supported if the file is partitioned or you use the same schema.
# For a robust solution, use pyarrow directly:

import pyarrow as pa
import pyarrow.parquet as pq

def append_df_to_parquet(df, parquet_path):
    table = pa.Table.from_pandas(df)
    # If the file does not exist, create it
    try:
        pq.write_to_dataset(table, root_path=parquet_path, append=True)
    except Exception:
        # If not a directory-based dataset, fallback to row-group append
        if not os.path.exists(parquet_path):
            pq.write_table(table, parquet_path)
        else:
            # Append as new row group (pyarrow >= 0.17)
            with pq.ParquetWriter(parquet_path, table.schema, use_dictionary=True, compression='snappy') as writer:
                writer.write_table(table)

In [13]:
# Robust Parquet append: handle corrupt or missing file gracefully.
import pandas as pd
from fastparquet import write, ParquetFile
import os

parquet_path = r'C:\Users\ChristopherCato\OneDrive - clarity-dx.com\code\bph\workcomp-rates\core\data\commercial_rates_GA.parquet'

# Try to load existing columns; if file is corrupt or unreadable, create new file
try:
    pf = ParquetFile(parquet_path)
    existing_cols = pf.columns

    # Add missing columns to output, fill with pd.NA
    for col in existing_cols:
        if col not in output.columns:
            output[col] = pd.NA

    # Remove extra columns not in existing file
    output_aligned = output[existing_cols]

    # Coerce all object columns to string to avoid fastparquet byte errors
    for col in output_aligned.columns:
        if output_aligned[col].dtype == "object":
            output_aligned[col] = output_aligned[col].astype(str)

    # Now append
    write(parquet_path, output_aligned, append=True)
except Exception as e:
    print(f"Could not read existing Parquet file, will overwrite. Reason: {e}")
    # Coerce all object columns to string
    for col in output.columns:
        if output[col].dtype == "object":
            output[col] = output[col].astype(str)
    # Write new file with all columns in output
    write(parquet_path, output, append=False)

Could not read existing Parquet file, will overwrite. Reason: 'NoneType' object is not iterable
